# Reforumlated Markowitz Model - Optimization Codes 

Import the necessary libraries

In [47]:
import numpy as np
import pandas as pd
import cvxpy as cp  
import cvxopt
import time

### Generating a single portfolio

Importing the Data

In [48]:
df = pd.read_excel("Daily_Returns.xlsx")

Adding the date as an index

In [49]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

Renaming the DataFrame

In [50]:
df_returns=df

Calculating the expected returns and the variance covariance matrix

In [51]:
# Calculate annualized average return for each stock. 
expected_returns = np.mean(df_returns,axis=0)*252

# Calculate the variance-covariance matrix
cov_matrix = df_returns.cov()


#Converting the expected returns to a numpy array
expected_returns = expected_returns.values

#The number of assets
n_assets = len(expected_returns)


Setting the rest of the constraints for the reforumated problem

In [52]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 0.1

# Maximum units of each asset that can be purchased (D)
D = 150

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 15

# The number of shares the investor would like to purchase
S = 1000

In [53]:
# Variables to solve for
x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase
y = cp.Variable(n_assets, boolean=True)  # Binary variable for asset inclusion

In [54]:
# Creating the weights
w = x/S

Setting up the optimization problem

In [55]:

# Constraints
risk_constraint = cp.quad_form(w, cov_matrix) <=  R**2  # Risk constraint
diversification_constraints = [x[i] <= D*y[i] for i in range(n_assets)]  # Diversification and linking
cardinality_constraint = cp.sum(y) <= K  # Cardinality constraint
non_negativity_constraints = [x >= 0]  # Ensure x is non-negative 
total_sum_constraint = [cp.sum(x)==S] #Ensure that the sum of shares is S

# Objective function to maximize total expected return
objective = cp.Maximize(expected_returns @ w)

# Problem definition
problem = cp.Problem(objective, [risk_constraint, cardinality_constraint] + non_negativity_constraints +diversification_constraints + total_sum_constraint )

Solving the optimization problem

In [56]:
# Solving the problem
problem.solve(solver=cp.CPLEX)


# Results
print("Status:", problem.status)
print("Maximum Expected Return:", problem.value)
print("Units of each asset to purchase:", x.value)
print("Assets included in the portfolio:", y.value)

Status: optimal
Maximum Expected Return: 0.3060180081039116
Units of each asset to purchase: [ -0. 150.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0. 150.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0.  -0.  -0.  -0.  -0. 150.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.
 100.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0.  -0. 150.  -0.  -0.  -0.
  -0.  -0.  -0.  -0. 150.  -0.  -0.  -0.  -0.  -0.  -0.  -0. 150.  -0.
  -0.  -0.  -0.  -0.  -0.]
Assets included in the portfolio: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Now let us test to see if all the constraints are satisfied

In [57]:
# Convert x.value and y.value to numpy arrays for easy manipulation
x_optimized = np.array(x.value)
y_optimized = np.array(y.value)
w_optimized = np.array(x_optimized/S)

# Constraint 1: Risk constraint
risk_value = np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized)
print(f"Risk Constraint Satisfied: {np.sqrt(risk_value) <= R}")
print(f"Risk Value: {np.sqrt(risk_value)} <= {R}")

# Constraint 2: Diversification and linking constraint
diversification_values = x_optimized <= D * y_optimized
print(f"Diversification and Linking Constraints Satisfied: {all(diversification_values)}")

# Constraint 3: Cardinality constraint
cardinality_value = (x_optimized > 0).sum()
print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}") 

# Constraint 5: sum of shares constraint
total_share = np.sum(x_optimized)
print(f"Sum of units Constraint Satisfied: {total_share == S}")
print(f"Number of Units Included: {total_share} == {S}")

Risk Constraint Satisfied: True
Risk Value: 0.01769587479509449 <= 0.1
Diversification and Linking Constraints Satisfied: True
Cardinality Constraint Satisfied: True
Number of Assets Included: 7 <= 15
Non-negativity Constraints Satisfied: True
Sum of units Constraint Satisfied: True
Number of Units Included: 1000.0 == 1000


### Generating multiple portfolios and the efficient frontier

In [58]:
"""
For this purpose, all the constraints except for the non negativity constraint,
sum of shares constraint and the risk constraint will be dropped so as to compare
the results being obtained with the vanilla markowits model
"""

#Setting the total number of shares to purchase
S=1000

Finding the portfolio with maximum return

In [59]:
"""
For this portfolio with the maximum return
the risk constraint will also be dropped
"""

# Variables to solve for
x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase


# Creating the weights
w = x/S

#Adding the sum constraint
total_sum_constraint = [cp.sum(x)==S]

# Objective function to maximize total expected return
objective = cp.Maximize(expected_returns @ w)

non_negativity_constraints = [x >= 0]  # Ensure x is non-negative 

# Problem definition
problem = cp.Problem(objective, total_sum_constraint+ non_negativity_constraints)

# Solving the problem
problem.solve(solver=cp.CPLEX)


# Results
print("Status:", problem.status)
print("Maximum Expected Return:", problem.value)
print("Units of each asset to purchase:", x.value)


# Convert x.value to numpy arrays for easy manipulation
x_optimized = np.array(x.value)
w_optimized = np.array(x_optimized/S)

# Risk constraint
risk_value_maximum = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))
print(f"Risk Value: {risk_value_maximum}")

Status: optimal
Maximum Expected Return: 0.3953949881466527
Units of each asset to purchase: [  -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0. 1000.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.]
Risk Value: 0.033701640

Finding the portfolio with minimum risk

In [60]:
"""
For this purpose the objective function
will be to minimize the risk and the sum of shares constraint
along with the non negativity constraint will be the only ones kept
"""

# Variables to solve for
x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase

# Creating the weights
w = x/S

#Adding the sum of shares constraint
total_sum_constraint = [cp.sum(x)==S]


# Objective function to maximize total risk
objective = cp.Minimize(cp.quad_form(w, cov_matrix))

#Adding the non negativity constraint
non_negativity_constraints = [x >= 0]   

# Problem definition
problem = cp.Problem(objective, total_sum_constraint+ non_negativity_constraints)

# Solving the problem
problem.solve(solver=cp.CPLEX)


# Results
print("Status:", problem.status)
print("Maximum Expected Return:", problem.value)
print("Units of each asset to purchase:", x.value)


# Convert x.value to numpy arrays for easy manipulation
x_optimized = np.array(x.value)
w_optimized = np.array(x_optimized/S)

# Risk constraint
risk_value_minimum = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))
print(f"Risk Value: {risk_value_minimum}")

Status: optimal
Maximum Expected Return: 9.4690142814331e-05
Units of each asset to purchase: [  -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0. 1000.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.   -0.
   -0.]
Risk Value: 0.00973088

Finding the portfolio with a 1% risk

In [1]:
"""
It is not always possible to generate efficient results for
a integer programming problem using classicial computing. Hence,
the efficient frontier will not be generated, as even the code given
below to generate a portfolio with a risk value less than 1% takes
more than 1 hour to run. Thus, it is computationally not possible
to generate an efficient frontier for a risk value of 1% using CPLEX
"""

'\nIt is not always possible to generate efficient results for\na integer programming problem using classicial computing. Hence,\nthe efficient frontier will not be generated, as even the code given\nbelow to generate a portfolio with a risk value less than 1% takes\nmore than 1 hour to run. Thus, it is computationally not possible\nto generate an efficient frontier for a risk value of 1% using CPLEX\n'

In [64]:
# Variables to solve for
x = cp.Variable(n_assets, integer=True)  # Units of each asset to purchase


# Creating the weights
w = x/S

#Adding the sum constraint
total_sum_constraint = [cp.sum(x)==S]

# Objective function to maximize total expected return
objective = cp.Maximize(expected_returns @ w)

non_negativity_constraints = [x >= 0]  # Ensure x is non-negative 

#Risk Constraints
risk_constraint = [cp.quad_form(w, cov_matrix) <=  0.01**2]  # Risk constraint

# Problem definition
problem = cp.Problem(objective, total_sum_constraint+ non_negativity_constraints+risk_constraint)

# Solving the problem
problem.solve(solver=cp.CPLEX)



# Results
print("Status:", problem.status)
print("Maximum Expected Return:", problem.value)
print("Units of each asset to purchase:", x.value)


# Convert x.value to numpy arrays for easy manipulation
x_optimized = np.array(x.value)
w_optimized = np.array(x_optimized/S)

# Risk constraint
risk_value_1 = np.sqrt(np.dot(np.dot(w_optimized.T, cov_matrix), w_optimized))
print(f"Risk Value: {risk_value_1}")